In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
from sklearn.manifold import TSNE

from vae import VariationalAutoencoder
from modules import train_epoch, test_epoch, plot_ae_outputs


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

In [ ]:
dataset = torchvision.datasets.ImageFolder("../media/")
# dataset = torchvision.datasets.MNIST(root="/home/szaryvip/Downloads", train=True, download=True, transform=None)
# for idx, (img, _) in enumerate(dataset):
#     if idx > 200:
#         break
#     img.save('../media/unlabeled/{:05d}.jpg'.format(idx))
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset.transform = train_transform


train_loader = DataLoader(dataset)

In [ ]:
in_channels = len(dataset[0][0])
d = 128
lr = 1e-3 
model = VariationalAutoencoder(in_channels, d, device)
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
model.to(device)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
   train_loss = train_epoch(model,device,train_loader,optim)

In [ ]:
model.eval()
encoding = []
with torch.no_grad():
    for img, _ in train_loader:
        img.to(device)
        encoded = model.encoder(img) 
        encoding.append(encoded)

In [ ]:
encoding